<a href="https://colab.research.google.com/github/ghasemieh/Duplicate-Bug-Identification-System/blob/master/Phase_1_Load_data_Data_Preparation_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Phase-1: Load data and Data Preparation

In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [4]:
df.head()

,id,product,component,reporter,bug_status,resolution,priority,bug_severity,version,short_desc,opendate,dup_list,root_id,disc_id
0,2,Log4j -,Layout,bugzilla,CLOS,FIXE,P3,nor,unspe,Just testing the Boogzeela setup for log4j,2001-01-08,NaN,NaN,NaN
1,3,Log4j -,Appender,matthew_scully,RESO,FIXE,P3,nor,unspe,file sharing,2001-01-08,NaN,NaN,NaN
2,22,Apache h,All,greenrd,CLOS,WONT,P5,enh,2.0-H,Need hooks for user-defined error handling,2001-01-09,NaN,NaN,NaN
3,27,Log4j -,Layout,richard.mccarthy,RESO,INVA,P1,nor,1.0,Error in the formatting of the (%F:%L) layout ...,2001-01-10,NaN,NaN,NaN
4,29,Log4j -,Other,woge,RESO,FIXE,P3,nor,1.0,"Calling Category.error(Object, Throwable) thro...",2001-01-11,NaN,NaN,NaN


**Remove NaN from the short_desc**

In [5]:
df.isnull().sum()

id                  0
product             0
component           0
reporter            0
bug_status          0
resolution          0
priority            0
bug_severity        0
version             0
short_desc         58
opendate            0
dup_list        41531
root_id         38016
disc_id         38016
dtype: int64

In [6]:
df = df[['id','short_desc','dup_list']]
df.head()

,id,short_desc,dup_list
0,2,Just testing the Boogzeela setup for log4j,NaN
1,3,file sharing,NaN
2,22,Need hooks for user-defined error handling,NaN
3,27,Error in the formatting of the (%F:%L) layout ...,NaN
4,29,"Calling Category.error(Object, Throwable) thro...",NaN


In [7]:
df.shape

(44049, 3)

Remove Blank Records

In [8]:
blanks = []

for i,id,sd,dl in df.itertuples():
    if type(sd)==str:
        if sd.isspace():
            blanks.append(i)
    else:
        blanks.append(i)
len(blanks)  

58

In [9]:
df.drop(blanks,inplace=True)
df.isnull().sum()

id                0
short_desc        0
dup_list      41474
dtype: int64

**Similarity Methods:**

In [10]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [0]:
def to_string(df,id):
    s = df[df['id'] == id]
    s = s.short_desc.to_string(index=False)
    return s[1:]

In [14]:
s1 = to_string(df,136)
print(s1)
s2 = to_string(df,535)
print(s2)

Infinite loop in PrefixManager.java BugRat Rep...
deadlock in connection handler


**Remove Puntuations**

In [0]:
#Remove Puntuations
def remove_puntuation(str):
    import re
    return re.sub(r'[^\w\s]','',str)

In [0]:
s1 = remove_puntuation(s1)
s2 = remove_puntuation(s2)

**Remove Stop Words**

In [0]:
#Remove Stop Words
def remove_stop_word(sentense): # accept string
    s = []
    sentense = nlp(sentense)
    for token in sentense:
      if token.is_stop == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [0]:
doc1 = remove_stop_word(s1)

In [0]:
doc2 = remove_stop_word(s2)

In [20]:
print('Similarity: ' + str(doc1.similarity(doc2)))

Similarity:0.42781221066187264


**Build a function of all previous work**

In [0]:
# Put all prevouse work together in a function
def prep(df,id):
    # Take an id and convert the short_desc to string
    def to_string(df,id):
        s = df[df['id'] == id]
        s = s.short_desc.to_string(index=False)
        return s[1:]

    s = to_string(df,id)

    # Remove Puntuations
    def remove_puntuation(str):
        import re
        return re.sub(r'[^\w\s]','',str)

    s = remove_puntuation(s)

    # Remove Stop Words
    def remove_stop_word(sentense): # accept string
        s = []
        sentense = nlp(sentense)
        for token in sentense:
            if token.is_stop == False:
                s.append(token) 
        # Convert list to string
        str1 = ""    
        for ele in s:  
            str1 += str(ele) +' '     
        return nlp(str1[:-1]) #return doc
    
    doc = remove_stop_word(s)

    return doc

In [0]:
# Prepare the vectors for each document
c = 0
prepared = []
for i,id,sd,db in df.itertuples():
    doc = prep(df,id)
    prepared.append((id,doc))
    c = c + 1
    #if c > 100:
    #    break

In [23]:
c = 0
for i in prepared:
    print(i[0], '\t', i[1].text)
    c = c + 1
    if c > 20:
        break

2 	 testing Boogzeela setup log4j
3 	 file sharing
22 	 Need hooks userdefined error handling
27 	 Error formatting FL layout
29 	 Calling CategoryerrorObject Throwable thro
31 	 Conversion log4j Priority types NT Event


In [0]:
# Calculate one by one similarity score
similarities = []
for i in prepared:
    for j in prepared:
        score = i[1].similarity(j[1])
        similarities.append((i[0],j[0],score))

In [1]:
len(similarities)

NameError: ignored